计算作者合作次数和机构合作次数
建立合作网络

In [1]:
import pandas as pd

df = pd.read_excel("../output_data/nodes_10y_clean.xlsx")


auPairs = {}
# 作者组合次数
auCoop = {}
# 作者合作次数

instPairs = {}
# 机构组合次数
instCoop = {}
# 机构合作次数

for i in range(len(df)):
    info = df.loc[i]
    ls = eval(info["AF"])
    aus = []
    insts = []
    for auInst in ls:
        for k, v in auInst.items():
            aus.append(k)
            insts.append(v)
    aus = list(set(aus))
    insts = list(set(insts))
    # 合作次数
    for au in aus:
        auCoop[au] = len(aus) - 1 + auCoop.get(au, 0)
    for inst in insts:
        instCoop[inst] = len(insts) - 1 + instCoop.get(inst, 0)
    # 合作组合次数
    for i in range(len(aus)):
        for j in range(i + 1, len(aus)):
            auPairs[(aus[i], aus[j])] = auPairs.get((aus[i], aus[j]), 0) + 1
    for i in range(len(insts)):
        for j in range(i + 1, len(insts)):
            instPairs[(insts[i], insts[j])] = instPairs.get((insts[i], insts[j]), 0) + 1

In [2]:
# 读入三种类型的节点信息
import json
with open("../output_data/jsonCluster_All2.json", "r", encoding="utf-8") as f:
    jsonData = json.load(f)  # 解析 JSON 文件

读入三种节点的信息 papernodes
au和inst添加合作次数——coops
建立link——pair、value
nodes+新link——合作网络json
更新总json——把nodes更新 
group==2 then coopnum=dc[id]

In [3]:
papaerNodes=[]
auNodes=[]
instNodes=[]
for node in jsonData["nodes"]:
    if node["group"]==1:
        papaerNodes.append(node)
    elif node["group"]==2:
        node["coop"]=auCoop[node["id"]]
        auNodes.append(node)
    else:
        node["coop"] = instCoop[node["id"]]
        instNodes.append(node)

print(papaerNodes[-1])
print(auNodes[-1])
print(instNodes[-1])
print(len(papaerNodes)+len(auNodes)+len(instNodes))

{'id': 'Vials: Visualizing Alternative Splicing of Genes', 'group': 1, 'doi': '10.1109/TVCG.2015.2467911', 'ab': 'Alternative splicing is a process by which the same DNA sequence is used to assemble different proteins, called protein isoforms. Alternative splicing works by selectively omitting some of the coding regions (exons) typically associated with a gene. Detection of alternative splicing is difficult and uses a combination of advanced data acquisition methods and statistical inference. Knowledge about the abundance of isoforms is important for understanding both normal processes and diseases and to eventually improve treatment through targeted therapies. The data, however, is complex and current visualizations for isoforms are neither perceptually efficient nor scalable. To remedy this, we developed Vials, a novel visual analysis tool that enables analysts to explore the various datasets that scientists use to make judgments about isoforms: the abundance of reads associated with

In [4]:
auLinks=[]
instLinks=[]
auClusterDc={}
instClusterDc={}
# 获取节点名称对应的社区号
for au in auNodes:
    auClusterDc[au["id"]]=au["cluster"]
for inst in instNodes:
    instClusterDc[inst["id"]]=inst["cluster"]
# 建立连边
for pair in auPairs:
    i=auClusterDc[pair[0]]
    j=auClusterDc[pair[1]]
    if i==j:
        link = {
            "source": pair[0],
            "target": pair[1],
            "value": auPairs[pair],
            "cluster": i,
        }
        auLinks.append(link)
for pair in instPairs:
    i = instClusterDc[pair[0]]
    j = instClusterDc[pair[1]]
    if i==j:
        link = {
            "source": pair[0],
            "target": pair[1],
            "value": instPairs[pair],
            "cluster": i,
        }
        instLinks.append(link)

In [5]:
# 建立合作网络
auCoopJson={}
auCoopJson["nodes"]=auNodes
auCoopJson["links"]=auLinks
instCoopJson={}
instCoopJson["nodes"]=instNodes
instCoopJson["links"]=instLinks

with open("../output_data/auCoop10.json", "w", encoding="utf-8") as f:
    json.dump(auCoopJson, f, indent=4, ensure_ascii=False)
with open("../output_data/instCoop10.json", "w", encoding="utf-8") as f:
    json.dump(instCoopJson, f, indent=4, ensure_ascii=False)

In [6]:
# 更新全图json
jsonLinks=jsonData["links"]
print(len(jsonLinks))
print(len(auNodes)+len(papaerNodes)+len(instNodes))


28435
11585


In [8]:
jsonData_Update={}
jsonData_Update["nodes"]=papaerNodes+auNodes+instNodes
jsonData_Update["links"]=jsonLinks
with open("../output_data/jsonCluster_All_Coop10.json", "w", encoding="utf-8") as f:
    json.dump(jsonData_Update, f, indent=4, ensure_ascii=False)

In [9]:
# 更新绘制图json 中的Coop
with open("../output_data/jsonCluster_Draw2.json", "r", encoding="utf-8") as f:
    jsonData_Draw = json.load(f)
print(len(jsonData_Draw["nodes"]))
print(len(jsonData_Draw["links"]))

2989
13567


In [10]:
auInstMax=[]    #最有影响力的作者和机构 用于绘制
for node in jsonData_Draw["nodes"]:
    if node["group"]==2 or node["group"]==3:
        auInstMax.append(node["id"])
DrawNodes=jsonData_Draw["nodes"]
for idx,node in enumerate(DrawNodes):
    if node["id"] in auInstMax:
        if node["group"]==2:
            DrawNodes[idx]["coop"]=auCoop[node["id"]]
        elif node["group"] == 3:
            DrawNodes[idx]["coop"] = instCoop[node["id"]]
print(DrawNodes[0])

{'id': 'Qu, Huamin', 'group': 2, 'paperCount': 65, 'inst': 'Hong Kong Univ Sci & Technol', 'cluster': 9, 'coop': 331}


In [11]:
print(len(DrawNodes))

2989


In [12]:
jsonData_Draw["nodes"]=DrawNodes
with open("../output_data/jsonCluster_Draw_Coop10.json", "w", encoding="utf-8") as f:
    json.dump(jsonData_Draw, f, indent=4, ensure_ascii=False)

In [14]:
# 验证
with open("../output_data/jsonCluster_Draw_Coop10.json", "r", encoding="utf-8") as f:
    jsonData_Draw = json.load(f)
print(len(jsonData_Draw["nodes"]))
print(len(jsonData_Draw["links"]))

2989
13567
